In [ ]:
import cv2
import numpy as np
import os
import pandas as pd

# フォルダ内の全画像ファイル名を取得
image_files = os.listdir('pokemon_drop')
image_files = [f for f in image_files if f.endswith('.png')]

# 結果を格納するためのリスト
results = []

for file in image_files:
    # 画像を読み込む
    pokemon = cv2.imread(f'pokemon_drop/{file}')
    pokemon_img = cv2.cvtColor(pokemon, cv2.COLOR_BGR2RGB)

    # 画像のグレースケール化
    img_gray = cv2.cvtColor(pokemon, cv2.COLOR_RGB2GRAY)

    # 白線部分の膨張
    kernel = np.ones((4,4),np.uint8)
    dilation = cv2.dilate(img_gray, kernel, iterations=1)

    # 膨張させた画像からグレーの画像を引く
    diff = cv2.subtract(dilation, img_gray)
    negaposi = 255 - diff

    # 輪郭内を黒色で塗りつぶし、黒色の画素数値を足す
    black_area = np.where(np.array(negaposi).flatten().reshape(-1) < 250, 1, 0).sum()

    # 画像全体のピクセル数から白色（または透明）のピクセル数を引く
    used_area = pokemon.shape[0] * pokemon.shape[1] - np.sum(pokemon == 255) / 3

    # 黒い場所 ÷ 輪郭内の線の部分
    poke_entropy = black_area / used_area

    # 結果をリストに追加
    results.append({'file': file, 'poke_entropy': poke_entropy})

# リストをデータフレームに変換
df = pd.DataFrame(results)

# データフレームを表示
print(df)